In [3]:
from pyspark.sql import SparkSession
import os
import configparser


In [4]:
config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read_file(open('dl.cfg'))

os.environ["AWS_ACCESS_KEY_ID"] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"] = config['AWS']['AWS_SECRET_ACCESS_KEY']

# Explore S3 path structure before doing Schema-on-Read

In [5]:
import boto3

In [6]:
s3_resrc = boto3.resource('s3',
                            region_name='us-west-2',
                            aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
                            aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])

In [7]:
s3_log_data = s3_resrc.Bucket("udacity-dend").objects.filter(Prefix="log-data/")
print("refer S3 done")

refer S3 done


In [8]:
for idx, _each in enumerate(s3_log_data):
    print(_each)
    if idx == 20:
        break

s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-01-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-02-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-03-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-04-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-05-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-06-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-07-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-08-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-09-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-10-events.json')
s3.ObjectSummary(b

In [9]:
s3_song_data = s3_resrc.Bucket("udacity-dend").objects.filter(Prefix="song-data/")
print("refer s3_song_data done")

refer s3_song_data done


In [10]:
for idx, _each in enumerate(s3_song_data):
    print(_each)
    if idx == 20:
        break

s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAK128F9318786.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAV128F421A322.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAABD128F429CF47.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAACN128F9355673.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEA128F935A30D.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAED128E0783FAB.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEM128F93347B9.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEW128F42930C0.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAFD128F92F423A.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAGR128F425B14B.json')
s3.ObjectSummary(

# Create Spark Session with Hadoop AWS package

In [11]:
spark = SparkSession.builder\
                    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
                    .getOrCreate()

print("Create Spark Session completed")

Create Spark Session completed


# Profiling data structure in each S3 Bucket

In [12]:
s3_bucket_root_path = "s3a://udacity-dend/"

song_data_root_path = s3_bucket_root_path + "song-data/"

song_data_postfix_path = "*/*/*/*.json"

log_data_root_path = s3_bucket_root_path + "log-data/"

log_data_postfix_path = "*/*/*.json"

song_data_full_path = os.path.join(s3_bucket_root_path,"song-data/",song_data_postfix_path)
log_data_full_path = os.path.join(s3_bucket_root_path, "log-data/",log_data_postfix_path)

song_data_sample_path = os.path.join(s3_bucket_root_path,"song-data/A/A/A/*.json")
log_data_sample_path = os.path.join(s3_bucket_root_path,"log-data/2018/11/*.json")                   

### song-data

In [13]:
print("Reading : ", song_data_sample_path)
song_df = spark.read.json(song_data_sample_path).drop_duplicates()
print("Finish reading JSON from S3")

Reading :  s3a://udacity-dend/song-data/A/A/A/*.json
Finish reading JSON from S3


In [14]:
song_df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [15]:
song_df.limit(10).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARJNIUY12298900C91,NaN,,NaN,Adelitas Way,213.94240,1,SOBLFFE12AF72AA5BA,Scream,2009
1,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music,189.57016,1,SOHKNRJ12A6701D1F8,Drop of Rain,0
2,AR0MWD61187B9B2B12,NaN,,NaN,International Noise Conspiracy,195.39546,1,SOHOZBI12A8C132E3C,Smash It Up,2000
3,ARCLYBR1187FB53913,37.54703,"San Mateo, CA",-122.31483,Neal Schon,304.56118,1,SOOVHYF12A8C134892,I'll Be Waiting,1989
4,ARGE7G11187FB37E05,NaN,"Brooklyn, NY",NaN,Cyndi Lauper,240.63955,1,SONRWUU12AF72A4283,Into The Nightlife,2008
5,ARKYKXP11F50C47A6A,NaN,,NaN,The Supersuckers,160.20853,1,SONQPZK12AB0182D84,Double Wide,0
6,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969
7,ARC1IHZ1187FB4E920,NaN,,NaN,Jamie Cullum,246.98730,1,SOXZYWX12A6310ED0C,It's About Time,0
8,AR9Q9YC1187FB5609B,NaN,New Jersey,NaN,Quest_ Pup_ Kevo,252.94322,1,SOFRDWL12A58A7CEF7,Hit Da Scene,0
9,ARSVTNL1187B992A91,51.50632,"London, England",-0.12714,Jonathan King,129.85424,1,SOEKAZG12AB018837E,I'll Slap Your Face (Entertainment USA Theme),2001


In [16]:
print(song_df.columns)
song_df.printSchema()

['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year']
root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



### Define schema for song

In [17]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField as Fld, DoubleType as Dbl, \
        StringType as Str, IntegerType as Int, DateType as Date, LongType as Long

In [18]:
song_schema = StructType([
    Fld("artist_id", Str()),
    Fld("artist_latitude", Dbl()),
    Fld("artist_location", Str()),
    Fld("artist_longitude", Dbl()),
    Fld("artist_name", Str()),
    Fld("duration", Dbl()),
    Fld("num_songs", Long()),
    Fld("song_id", Str()),
    Fld("title", Str()),
    Fld("year", Long()),
])

print("Reading : ", song_data_sample_path)
song_with_schema_df = spark.read.json(song_data_sample_path, schema=song_schema).drop_duplicates()
print("Finish reading JSON from S3")

Reading :  s3a://udacity-dend/song-data/A/A/A/*.json
Finish reading JSON from S3


In [19]:
song_with_schema_df.limit(10).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARJNIUY12298900C91,NaN,,NaN,Adelitas Way,213.94240,1,SOBLFFE12AF72AA5BA,Scream,2009
1,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music,189.57016,1,SOHKNRJ12A6701D1F8,Drop of Rain,0
2,AR0MWD61187B9B2B12,NaN,,NaN,International Noise Conspiracy,195.39546,1,SOHOZBI12A8C132E3C,Smash It Up,2000
3,ARCLYBR1187FB53913,37.54703,"San Mateo, CA",-122.31483,Neal Schon,304.56118,1,SOOVHYF12A8C134892,I'll Be Waiting,1989
4,ARGE7G11187FB37E05,NaN,"Brooklyn, NY",NaN,Cyndi Lauper,240.63955,1,SONRWUU12AF72A4283,Into The Nightlife,2008
5,ARKYKXP11F50C47A6A,NaN,,NaN,The Supersuckers,160.20853,1,SONQPZK12AB0182D84,Double Wide,0
6,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969
7,ARC1IHZ1187FB4E920,NaN,,NaN,Jamie Cullum,246.98730,1,SOXZYWX12A6310ED0C,It's About Time,0
8,AR9Q9YC1187FB5609B,NaN,New Jersey,NaN,Quest_ Pup_ Kevo,252.94322,1,SOFRDWL12A58A7CEF7,Hit Da Scene,0
9,ARSVTNL1187B992A91,51.50632,"London, England",-0.12714,Jonathan King,129.85424,1,SOEKAZG12AB018837E,I'll Slap Your Face (Entertainment USA Theme),2001


In [20]:
# songs_table
song_columns = ["song_id", "title", "artist_id", "year", "duration"]
songs_table = song_with_schema_df.select(song_columns)
songs_table.limit(10).toPandas()

,song_id,title,artist_id,year,duration
0,SOBLFFE12AF72AA5BA,Scream,ARJNIUY12298900C91,2009,213.94240
1,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
2,SOHOZBI12A8C132E3C,Smash It Up,AR0MWD61187B9B2B12,2000,195.39546
3,SOOVHYF12A8C134892,I'll Be Waiting,ARCLYBR1187FB53913,1989,304.56118
4,SONRWUU12AF72A4283,Into The Nightlife,ARGE7G11187FB37E05,2008,240.63955
5,SONQPZK12AB0182D84,Double Wide,ARKYKXP11F50C47A6A,0,160.20853
6,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546
7,SOXZYWX12A6310ED0C,It's About Time,ARC1IHZ1187FB4E920,0,246.98730
8,SOFRDWL12A58A7CEF7,Hit Da Scene,AR9Q9YC1187FB5609B,0,252.94322
9,SOEKAZG12AB018837E,I'll Slap Your Face (Entertainment USA Theme),ARSVTNL1187B992A91,2001,129.85424


In [21]:
#artists_table
artist_columns = ["artist_id", "artist_name as name", "artist_location as location", "artist_longitude as longitude", "artist_latitude as latitude"]
artists_table = song_df.selectExpr(artist_columns)
artists_table.limit(10).toPandas()

,artist_id,name,location,longitude,latitude
0,ARJNIUY12298900C91,Adelitas Way,,NaN,NaN
1,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
2,AR0MWD61187B9B2B12,International Noise Conspiracy,,NaN,NaN
3,ARCLYBR1187FB53913,Neal Schon,"San Mateo, CA",-122.31483,37.54703
4,ARGE7G11187FB37E05,Cyndi Lauper,"Brooklyn, NY",NaN,NaN
5,ARKYKXP11F50C47A6A,The Supersuckers,,NaN,NaN
6,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",-90.04892,35.14968
7,ARC1IHZ1187FB4E920,Jamie Cullum,,NaN,NaN
8,AR9Q9YC1187FB5609B,Quest_ Pup_ Kevo,New Jersey,NaN,NaN
9,ARSVTNL1187B992A91,Jonathan King,"London, England",-0.12714,51.50632


### log-data

In [22]:
print("Reading : ", log_data_sample_path)
log_data_df = spark.read.json(log_data_sample_path)
print("Finish reading log_data_df JSON from S3")

Reading :  s3a://udacity-dend/log-data/2018/11/*.json
Finish reading log_data_df JSON from S3


In [23]:
log_data_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [24]:
log_data_df.limit(10).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20...,12
5,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,597,Blackbird,200,1542253449796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
6,None,Logged In,Samuel,M,1,Gonzalez,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,About,1.540493e+12,597,None,200,1542253460796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
7,None,Logged Out,None,None,0,None,NaN,paid,None,PUT,Login,NaN,602,None,307,1542260074796,None,
8,None,Logged In,Tegan,F,1,Levine,NaN,paid,"Portland-South Portland, ME",GET,Home,1.540794e+12,602,None,200,1542260277796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
9,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,602,Best Of Both Worlds (Remastered Album Version),200,1542260935796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


# Define schema for log

In [25]:
print(log_data_df.columns)
print("Current Schema")
log_data_df.printSchema()

['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId']
Current Schema
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [26]:
from pyspark.sql.types import StructType, StructField as Fld, DoubleType as Dbl, \
        StringType as Str, IntegerType as Int, DateType as Date, LongType as Long

log_schema = StructType([
    Fld("artist", Str()),
    Fld("auth", Str()),
    Fld("firstName", Str()),
    Fld("gender", Str()),
    Fld("itemInSession", Long()),
    Fld("lastName", Str()),
    Fld("length", Dbl()),
    Fld("level", Str()),
    Fld("location", Str()),
    Fld("method", Str()),
    Fld("page", Str()),
    Fld("registration", Dbl()),
    Fld("sessionId", Long()),
    Fld("song", Str()),
    Fld("status", Int()),
    Fld("ts", Long()),
    Fld("userAgent", Str()),
    Fld("userId", Str())
])

print("Reading : ", log_data_sample_path)
log_data_with_schema_df = spark.read.json(log_data_sample_path, schema=log_schema).drop_duplicates()
print("Finish reading log_data_df JSON from S3")


Reading :  s3a://udacity-dend/log-data/2018/11/*.json
Finish reading log_data_df JSON from S3


In [27]:
print("Current Schema")
log_data_with_schema_df.printSchema()

Current Schema
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [28]:
log_data_with_schema_df.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Avery,F,7,Watkins,304.84853,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.540872e+12,324,Uprising,200,1542276912796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Marina And The Diamonds,Logged In,Avery,F,56,Watkins,182.59546,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.540872e+12,324,Oh No! (album version),200,1542288870796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
2,Kanye West / Consequence / Cam'Ron,Logged In,Aleena,F,69,Kirby,333.34812,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,619,Gone,200,1542320503796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,Pavement,Logged In,Kynnedi,F,0,Sanchez,99.16036,free,"Cedar Rapids, IA",PUT,NextSong,1.541079e+12,631,Mercy:The Laundromat,200,1542783174796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5...",89
4,Il Divo,Logged In,Kate,F,35,Harrell,229.87710,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,817,Unchained Melody (Senza Catene),200,1542828982796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


In [29]:
# users_table
user_columns = ["userId", "firstName", "lastName", "gender", "level"]
users_table = log_data_with_schema_df.select(user_columns)
users_table.limit(5).toPandas()

,userId,firstName,lastName,gender,level
0,30,Avery,Watkins,F,paid
1,30,Avery,Watkins,F,paid
2,44,Aleena,Kirby,F,paid
3,89,Kynnedi,Sanchez,F,free
4,97,Kate,Harrell,F,paid


### create UDF functions for log-related tables construction

In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType
from datetime import datetime


get_timestamp = udf(lambda x: datetime.utcfromtimestamp(x/1000.0), TimestampType())
get_datetime = udf(lambda x: datetime.utcfromtimestamp(x/1000.0).strftime('%Y-%m-%d %H:%M:%S'))

In [31]:
# get only NextSong
ns_log_data_with_schema_df = log_data_with_schema_df.filter(log_data_with_schema_df.page == "NextSong")

In [32]:
ns_log_data_with_schema_df = ns_log_data_with_schema_df.withColumn("ts_datetime", get_datetime("ts"))
ns_log_data_with_schema_df = ns_log_data_with_schema_df.withColumn("ts_timestamp", get_timestamp("ts"))


In [33]:
ns_log_data_with_schema_df.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,ts_datetime,ts_timestamp
0,Muse,Logged In,Avery,F,7,Watkins,304.84853,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.540872e+12,324,Uprising,200,1542276912796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,2018-11-15 10:15:12,2018-11-15 10:15:12.796
1,Marina And The Diamonds,Logged In,Avery,F,56,Watkins,182.59546,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.540872e+12,324,Oh No! (album version),200,1542288870796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,2018-11-15 13:34:30,2018-11-15 13:34:30.796
2,Kanye West / Consequence / Cam'Ron,Logged In,Aleena,F,69,Kirby,333.34812,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,619,Gone,200,1542320503796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44,2018-11-15 22:21:43,2018-11-15 22:21:43.796
3,Pavement,Logged In,Kynnedi,F,0,Sanchez,99.16036,free,"Cedar Rapids, IA",PUT,NextSong,1.541079e+12,631,Mercy:The Laundromat,200,1542783174796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5...",89,2018-11-21 06:52:54,2018-11-21 06:52:54.796
4,Il Divo,Logged In,Kate,F,35,Harrell,229.87710,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,817,Unchained Melody (Senza Catene),200,1542828982796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,2018-11-21 19:36:22,2018-11-21 19:36:22.796


In [34]:
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek

ns_log_data_with_schema_df= ns_log_data_with_schema_df.withColumn("hour", hour("ts_timestamp")) \
                            .withColumn("day", dayofmonth("ts_timestamp")) \
                            .withColumn("week", weekofyear("ts_timestamp")) \
                            .withColumn("month", month("ts_timestamp")) \
                            .withColumn("year", year("ts_timestamp")) \
                            .withColumn("weekday", dayofweek("ts_timestamp"))

In [35]:
ns_log_data_with_schema_df.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,userAgent,userId,ts_datetime,ts_timestamp,hour,day,week,month,year,weekday
0,Muse,Logged In,Avery,F,7,Watkins,304.84853,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,2018-11-15 10:15:12,2018-11-15 10:15:12.796,10,15,46,11,2018,5
1,Marina And The Diamonds,Logged In,Avery,F,56,Watkins,182.59546,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,2018-11-15 13:34:30,2018-11-15 13:34:30.796,13,15,46,11,2018,5
2,Kanye West / Consequence / Cam'Ron,Logged In,Aleena,F,69,Kirby,333.34812,paid,"Waterloo-Cedar Falls, IA",PUT,...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44,2018-11-15 22:21:43,2018-11-15 22:21:43.796,22,15,46,11,2018,5
3,Pavement,Logged In,Kynnedi,F,0,Sanchez,99.16036,free,"Cedar Rapids, IA",PUT,...,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_5...",89,2018-11-21 06:52:54,2018-11-21 06:52:54.796,6,21,47,11,2018,4
4,Il Divo,Logged In,Kate,F,35,Harrell,229.87710,paid,"Lansing-East Lansing, MI",PUT,...,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,2018-11-21 19:36:22,2018-11-21 19:36:22.796,19,21,47,11,2018,4


In [36]:
# time_table
time_column = ["ts", "hour", "day", "week", "month", "year", "weekday"]
time_table = ns_log_data_with_schema_df.select(time_column).drop_duplicates()
time_table.limit(5).toPandas()

,ts,hour,day,week,month,year,weekday
0,1542729770796,16,20,47,11,2018,3
1,1543085795796,18,24,47,11,2018,7
2,1543441920796,21,28,48,11,2018,4
3,1542795222796,10,21,47,11,2018,4
4,1541530372796,18,6,45,11,2018,3


### Create a songplays_table
This table is created from a join between song_table and log_table

In [37]:
print(ns_log_data_with_schema_df.columns)
print("===")
print(song_with_schema_df.columns)

['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId', 'ts_datetime', 'ts_timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday']
===
['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year']


In [45]:
song_with_schema_df.createOrReplaceTempView("song_df")

spark.sql("""
    SELECT count(*)
    FROM song_df
""").show()

+--------+
|count(1)|
+--------+
|      24|
+--------+



In [47]:
song_with_schema_df.write \
    .mode('overwrite') \
    .partitionBy('year', 'artist_name') \
    .parquet("data/data_outputs/songs/")

print("Finished exporting song parquet ")

Finished exporting song parquet 


In [51]:
parquet_path = "data/data_outputs/songs/*/*/*.parquet"
parquet_test = spark.read.parquet(parquet_path)
print('reading completed')


reading completed


In [52]:
parquet_test.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)



In [39]:
ns_log_data_with_schema_df.createOrReplaceTempView("log_df")
song_with_schema_df.createOrReplaceTempView("song_df")

spark.sql("""
    SELECT song, artist, s.artist_name
    FROM log_df as l
    INNER JOIN song_df as s
    ON s.title = l.song
    LIMIT 10
""").show()

+------+---------------+------------+
|  song|         artist| artist_name|
+------+---------------+------------+
|Scream|Michael Jackson|Adelitas Way|
+------+---------------+------------+



In [40]:
song_with_schema_df.where(song_with_schema_df.title == "Scream").show()

+------------------+---------------+---------------+----------------+------------+--------+---------+------------------+------+----+
|         artist_id|artist_latitude|artist_location|artist_longitude| artist_name|duration|num_songs|           song_id| title|year|
+------------------+---------------+---------------+----------------+------------+--------+---------+------------------+------+----+
|ARJNIUY12298900C91|           null|               |            null|Adelitas Way|213.9424|        1|SOBLFFE12AF72AA5BA|Scream|2009|
+------------------+---------------+---------------+----------------+------------+--------+---------+------------------+------+----+



In [41]:
ns_log_data_with_schema_df.where(ns_log_data_with_schema_df.song == "Scream").show()

+---------------+---------+---------+------+-------------+--------+-------+-----+--------------------+------+--------+-----------------+---------+------+------+-------------+--------------------+------+-------------------+--------------------+----+---+----+-----+----+-------+
|         artist|     auth|firstName|gender|itemInSession|lastName| length|level|            location|method|    page|     registration|sessionId|  song|status|           ts|           userAgent|userId|        ts_datetime|        ts_timestamp|hour|day|week|month|year|weekday|
+---------------+---------+---------+------+-------------+--------+-------+-----+--------------------+------+--------+-----------------+---------+------+------+-------------+--------------------+------+-------------------+--------------------+----+---+----+-----+----+-------+
|Michael Jackson|Logged In|     Kate|     F|           32| Harrell|278.282| paid|Lansing-East Lans...|   PUT|NextSong|1.540472624796E12|      605|Scream|   200|154229874

In [42]:

joined_song_log_df = ns_log_data_with_schema_df.join(song_with_schema_df ,
                                                    [song_with_schema_df.title == ns_log_data_with_schema_df.song ,
                                                    song_with_schema_df.artist_name == ns_log_data_with_schema_df.artist],
                                                    how='inner')


In [43]:
joined_song_log_df.limit(5).show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+-----------+------------+----+---+----+-----+----+-------+---------+---------------+---------------+----------------+-----------+--------+---------+-------+-----+----+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|ts_datetime|ts_timestamp|hour|day|week|month|year|weekday|artist_id|artist_latitude|artist_location|artist_longitude|artist_name|duration|num_songs|song_id|title|year|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+-----------+------------+----+---+----+-----+----+-------+---------+---------------+---------------+----------------+-----------+--------+---------+-------+-----+----+
+------+----+---------+------+-------------+--------+------+-----+-

In [44]:
# songplays_table

from pyspark.sql.functions import monotonically_increasing_id

songplays_column = [monotonically_increasing_id().alias('songplay_id'), "ts", "userId", "level", "song_id", "artist_id", "sessionId", "location", "userAgent"]
songplays_table = joined_song_log_df.select(songplays_column)
songplays_table.limit(5).show()


+-----------+---+------+-----+-------+---------+---------+--------+---------+
|songplay_id| ts|userId|level|song_id|artist_id|sessionId|location|userAgent|
+-----------+---+------+-----+-------+---------+---------+--------+---------+
+-----------+---+------+-----+-------+---------+---------+--------+---------+

